problem: Design a Predictive Parser for the following grammar
             G: {E-> TE’, E’ -> +TE’ | 0, T-> FT’, T’-> *FT’|0, F-> (E) | id}.

In [8]:
#Some helper functions
def print_iter(Matched,Stack,Input,Action,verbose=True):
    if verbose==True:
        print(".".join(Matched).ljust(30)," | ",".".join(Stack).ljust(25)," | ",".".join(Input).ljust(30)," | ",Action)
#The predictive parsing algorithm
def predictive_parsing(sentence,parsingtable,terminals,start_state="S",verbose=True):      #Set verbose to false to not see the stages of the algorithm
    status = None
    match = []
    stack = [start_state,"$"]
    Inp = sentence.split(".")
    if verbose==True:
        print_iter(["Matched"],["Stack"],["Input"],"Action")
    print_iter(match,stack,Inp,"Initial",verbose)
    action=[]
    while(len(sentence)>0 and status!=False):
        top_of_input = Inp[0]
        pos = top_of_input
        if stack[0] =="$" and pos == "$" :
            print_iter(match,stack,Inp,"Accepted",verbose)
            return "Accepted"
        if stack[0] == pos:
            print_iter(match,stack,Inp,"Pop",verbose)
            match.append(stack[0])
            del(stack[0])
            del(Inp[0])
            continue
        if stack[0]=="epsilon":
            print_iter(match,stack,Inp,"Poping Epsilon",verbose)
            del(stack[0])
            continue
        try:
            production=parsingtable[stack[0]][pos]
            print_iter(match,stack,Inp,stack[0]+" -> "+production,verbose)
        except:
            return "error for "+str(stack[0])+" on "+str(pos),"Not Accepted"

        new = production.split(".")   
        stack=new+stack[1:]
    return "Not Accepted"

if __name__=="__main__":
    #Example for the working of the predictive parsing :-
    #input for the grammar : E->TE1;E1->+TE1|epsilon;T->FT1 ...
    parsingtable = {
    "E" : {"id" : "T.E1", "(" : "T.E1"},
    "E1" : {"+":"+.T.E1", ")":"epsilon", "$" : "epsilon"},
    "T" : {"id" : "F.T1", "(" : "F.T1" },
    "T1" : {"+" : "epsilon", "*" : "*.F.T1", ")" : "epsilon", "$" : "epsilon"},
    "F":{"id":"id","(":"(.E.)"}
    }
    terminals = ["id","(",")","+","*"]
    print(predictive_parsing(sentence="id.+.(.id.+.id.).$",parsingtable=parsingtable,terminals=terminals,start_state="E",verbose=True))
    #Another Example done in class:-
    print(predictive_parsing(sentence="c.c.c.c.d.d.$",parsingtable={"S" : {"c":"C.C","d":"C.C"},"C":{"c":"c.C","d":"d"}},terminals=["c,d"],start_state="S"))

Matched                         |  Stack                      |  Input                           |  Action
                                |  E.$                        |  id.+.(.id.+.id.).$              |  Initial
                                |  E.$                        |  id.+.(.id.+.id.).$              |  E -> T.E1
                                |  T.E1.$                     |  id.+.(.id.+.id.).$              |  T -> F.T1
                                |  F.T1.E1.$                  |  id.+.(.id.+.id.).$              |  F -> id
                                |  id.T1.E1.$                 |  id.+.(.id.+.id.).$              |  Pop
id                              |  T1.E1.$                    |  +.(.id.+.id.).$                 |  T1 -> epsilon
id                              |  epsilon.E1.$               |  +.(.id.+.id.).$                 |  Poping Epsilon
id                              |  E1.$                       |  +.(.id.+.id.).$                 |  E1 -> +.T.E1
id         

In [9]:
def numr(c):
    switcher = {
        'S': 0,
        'A': 1,
        'B': 2,
        'C': 3,
        'a': 0,
        'b': 1,
        'c': 2,
        'd': 3,
        '$': 4
    }
    return switcher.get(c, 2)

def main():
    table = [[' ' for _ in range(6)] for _ in range(5)]
    
    prol = ["S", "A", "A", "B", "B", "C", "C"]
    pror = ["A", "Bb", "Cd", "aB", "@", "Cc", "@"]
    prod = ["S->A", "A->Bb", "A->Cd", "B->aB", "B->@", "C->Cc", "C->@"]
    first = ["abcd", "ab", "cd", "a@", "@", "c@", "@"]
    follow = ["$", "$", "$", "a$", "b$", "c$", "d$"]

    print("\nThe following is the predictive parsing table for the following grammar:")
    for p in prod:
        print(p)

    print("\nPredictive parsing table is")
    for i in range(7):
        k = min(len(first[i]), 10)  # Use the minimum length
        for j in range(k):
            if first[i][j] != '@':
                table[numr(prol[i][0]) + 1][numr(first[i][j]) + 1] = prod[i]

    for i in range(7):
        if len(pror[i]) == 1:
            if pror[i][0] == '@':
                k = len(follow[i])
                for j in range(k):
                    table[numr(prol[i][0]) + 1][numr(follow[i][j]) + 1] = prod[i]

    table[0][0] = " "
    table[0][1] = "a"
    table[0][2] = "b"
    table[0][3] = "c"
    table[0][4] = "d"
    table[0][5] = "$"
    table[1][0] = "S"
    table[2][0] = "A"
    table[3][0] = "B"
    table[4][0] = "C"

    print("\n" + "-" * 56)
    for i in range(5):
        for j in range(6):
            print(f"| {table[i][j]:^10}", end="")
            if j == 5:
                print("|")
                print("-" * 56)

if __name__ == "__main__":
    main()


The following is the predictive parsing table for the following grammar:
S->A
A->Bb
A->Cd
B->aB
B->@
C->Cc
C->@

Predictive parsing table is

--------------------------------------------------------
|           |     a     |     b     |     c     |     d     |     $     |
--------------------------------------------------------
|     S     |    S->A   |    S->A   |    S->A   |    S->A   |           |
--------------------------------------------------------
|     A     |   A->Bb   |   A->Bb   |   A->Cd   |   A->Cd   |           |
--------------------------------------------------------
|     B     |   B->aB   |    B->@   |           |           |    B->@   |
--------------------------------------------------------
|     C     |           |           |   C->Cc   |    C->@   |    C->@   |
--------------------------------------------------------


In [10]:
# Predictive Parser for the given grammar
def predictive_parser(input_str):
    global input_index, current_token
    input_index = 0
    current_token = ""

    # Get the first token
    get_next_token()

    # Start parsing from the initial non-terminal
    if E():
        if current_token == '$':
            print("String accepted by the grammar.")
        else:
            print("String not fully parsed.")
    else:
        print("String not accepted by the grammar.")

def get_next_token():
    global input_index, current_token
    if input_index < len(input_str):
        current_token = input_str[input_index]
        input_index += 1
    else:
        current_token = '$'  # End of input marker

# Grammar rules
def E():
    print("E -> TE'")
    if T():
        if E_prime():
            return True
    return False

def E_prime():
    global current_token
    if current_token == '+':
        print("E' -> +TE'")
        get_next_token()
        if T():
            if E_prime():
                return True
    else:
        print("E' -> 0")
    return True

def T():
    print("T -> FT'")
    if F():
        if T_prime():
            return True
    return False

def T_prime():
    global current_token
    if current_token == '*':
        print("T' -> *FT'")
        get_next_token()
        if F():
            if T_prime():
                return True
    else:
        print("T' -> 0")
    return True

def F():
    global current_token
    if current_token == '(':
        print("F -> (E)")
        get_next_token()
        if E():
            if current_token == ')':
                get_next_token()
                return True
    elif current_token.isalpha() or current_token.isdigit():
        print("F -> id")
        get_next_token()
        return True
    return False

# Example usage
input_str = "id * (id + id)"
predictive_parser(input_str)

E -> TE'
T -> FT'
F -> id
T' -> 0
E' -> 0
String not fully parsed.
